In [1]:
from sympy import *
init_printing() #for pretty printing of symbolic maniputlation output in latex

In [ ]:
x = Symbol('x')
y = Symbol('y')

In [ ]:
expand((x+y)**3)

In [1]:
test = (2,3)

In [2]:
test[0]

2